In [8]:
import sys
import os
import glob
import subprocess

# Nombre del entorno uv ya existente
VENV_DIR = ".venv"

# Validacion de existencia del entorno
if not os.path.exists(VENV_DIR):
    raise FileNotFoundError(f"No se encuentra {VENV_DIR}. Ejecuta 'uv venv' en terminal.")

# Inyeccion de site-packages al sistema
# Esto permite usar rapidfuzz/pandas instalados via uv
site_packages = glob.glob(f"{VENV_DIR}/lib/python*/site-packages")
if site_packages:
    site_pkg = os.path.abspath(site_packages[0])
    if site_pkg not in sys.path:
        sys.path.insert(0, site_pkg)
        print(f"Librerias cargadas desde: {site_pkg}")
else:
    raise FileNotFoundError("No se detecto carpeta de librerias en .venv")

In [9]:
import pandas as pd
import numpy as np
import os
import gc
from rapidfuzz import process, fuzz
from tqdm.notebook import tqdm

print("Modulos importados correctamente.")

Modulos importados correctamente.


In [10]:
# Rutas
ARCHIVO_INPUT = 'nombres.csv'
ARCHIVOS_BUSQUEDA = ['ruts.csv']

# Columnas input (Nombres separados)
COLS_NOMBRES_INPUT = ['name1', 'name2', 'lastname1', 'lastname2']

# Columnas búsqueda (Configuradas según tu archivo ruts.csv)
COL_NOMBRE_BUSQUEDA = 'nombre' 
COL_RUT_BUSQUEDA = 'rut'

# Columnas adicionales (Ajusta esto si ruts.csv tiene más datos como edad o dirección)
# Si no estás seguro, deja la lista vacía para evitar errores: []
COLUMNAS_EXTRA = [] 

UMBRAL = 85
ARCHIVO_SALIDA = 'resultado_cruce_final.xlsx'

In [13]:
def cargar_optimo(ruta, cols_necesarias=None):
    if not os.path.exists(ruta):
        raise FileNotFoundError(f"No existe: {ruta}")
    
    # 1. Detección automática del separador
    separadores = [';', ',', '\t'] 
    sep_ganador = None
    cols_max = 0
    
    # Testeo rápido de separador
    for sep in separadores:
        try:
            # Probamos leer con python engine que es tolerante
            df_temp = pd.read_csv(ruta, sep=sep, nrows=5, on_bad_lines='skip', engine='python')
            if len(df_temp.columns) > cols_max:
                cols_max = len(df_temp.columns)
                sep_ganador = sep
        except:
            continue
    
    if sep_ganador is None: sep_ganador = ';'
    
    print(f"-> Leyendo {os.path.basename(ruta)} | Separador: '{sep_ganador}'")

    # 2. Carga con Prioridad de Encoding (UTF-8 primero para arreglar la Ñ)
    df = None
    
    # Intento A: UTF-8 (Estándar moderno, arregla símbolos raros)
    try:
        df = pd.read_csv(
            ruta, 
            sep=sep_ganador, 
            encoding='utf-8',  # <--- CAMBIO CLAVE
            low_memory=False, 
            on_bad_lines='skip'
        )
    except:
        pass
        
    # Intento B: Latin-1 (Si UTF-8 falla, usamos este)
    if df is None:
        try:
            df = pd.read_csv(
                ruta, 
                sep=sep_ganador, 
                encoding='latin-1', 
                low_memory=False, 
                on_bad_lines='skip'
            )
        except:
            # Intento C: Motor Python (Último recurso para archivos rotos)
            df = pd.read_csv(
                ruta, 
                sep=sep_ganador, 
                encoding='latin-1', 
                engine='python',
                index_col=False
            )

    # 3. Limpieza de cabeceras
    df.columns = df.columns.astype(str).str.strip().str.lower()
    
    return df

def limpiar_texto(serie):
    """
    Convierte a string, minúsculas, quita espacios Y normaliza ñ->n
    para asegurar que 'nuñez' haga match con 'nunez'
    """
    s = serie.fillna('').astype(str).str.lower().str.strip()
    # Reemplazo específico para mejorar el cruce sin dañar el dato original
    return s.str.replace('ñ', 'n', regex=False).str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')

In [15]:
# === CORRECCIÓN DE IMPORTACIÓN ===
# Usamos la versión estándar para evitar error de 'IProgress'
from tqdm import tqdm 
from rapidfuzz import process, fuzz
import gc

print("--- INICIANDO ---")

# 1. Carga Input
df_input = cargar_optimo(ARCHIVO_INPUT)
print(f"Filas leídas en Input: {len(df_input)}")

if df_input.empty:
    raise ValueError("El input sigue vacío. Revisa el log de arriba.")

# Generar llave input
print("Generando llaves...")
df_input['__key__'] = ""
cols_input_lower = [c.lower() for c in COLS_NOMBRES_INPUT]

if isinstance(cols_input_lower, list):
    for col in cols_input_lower:
        if col in df_input.columns:
            df_input['__key__'] += limpiar_texto(df_input[col]) + " "
else:
    col = cols_input_lower[0] if isinstance(cols_input_lower, list) else cols_input_lower
    df_input['__key__'] = limpiar_texto(df_input[col])
    
df_input['__key__'] = df_input['__key__'].str.strip()

# 2. Carga Maestra
print("Cargando Maestra...")
df_fuente = cargar_optimo(ARCHIVOS_BUSQUEDA[0])

# Buscar la columna correcta de nombre en la fuente
col_nombre_real = None
col_busqueda_lower = COL_NOMBRE_BUSQUEDA.lower()

if col_busqueda_lower in df_fuente.columns:
    col_nombre_real = col_busqueda_lower
else:
    posibles = [c for c in df_fuente.columns if 'nombre' in c]
    if posibles:
        col_nombre_real = posibles[0]
        print(f"Aviso: Usando columna '{col_nombre_real}' para nombres.")

if not col_nombre_real:
    raise ValueError(f"No encontré columna de nombres en Maestra. Disponibles: {list(df_fuente.columns)}")

# Preparar Maestra
print("Indexando maestra...")
df_fuente['__key__'] = limpiar_texto(df_fuente[col_nombre_real])
df_fuente = df_fuente[df_fuente['__key__'] != '']
df_fuente_unicos = df_fuente.drop_duplicates(subset=['__key__'])

opciones = df_fuente_unicos['__key__'].tolist()
mapa = pd.Series(df_fuente_unicos.index.values, index=df_fuente_unicos['__key__']).to_dict()

del df_fuente_unicos
gc.collect()

# 3. Cruce
print(f"Cruzando {len(df_input)} registros...")
resultados = {'RUT_ENCONTRADO': [], 'SCORE': [], 'NOMBRE_MATCH': []}
extras = {c: [] for c in COLUMNAS_EXTRA}

# TQDM estándar (barra de texto)
for nombre in tqdm(df_input['__key__'], desc="Progreso"):
    match_row = None
    score = 0
    nombre_match = None
    
    if nombre:
        res = process.extractOne(nombre, opciones, scorer=fuzz.token_sort_ratio, score_cutoff=UMBRAL)
        if res:
            nombre_match, score, _ = res
            idx = mapa[nombre_match]
            match_row = df_fuente.loc[idx]
            
    # Búsqueda insensible a mayúsculas para RUT
    rut_val = None
    if match_row is not None:
        for c in match_row.index:
            if 'rut' in c.lower() or 'run' in c.lower():
                rut_val = match_row[c]
                break
    
    resultados['RUT_ENCONTRADO'].append(rut_val)
    resultados['SCORE'].append(score)
    resultados['NOMBRE_MATCH'].append(nombre_match)
    
    for c in COLUMNAS_EXTRA:
        val = None
        if match_row is not None:
            for k in match_row.index:
                if c.lower() == k.lower():
                    val = match_row[k]
                    break
        extras[c].append(val)

# 4. Guardar
for k, v in resultados.items(): df_input[k] = v
for k, v in extras.items(): df_input[k] = v
df_input.drop(columns=['__key__'], inplace=True, errors='ignore')

if ARCHIVO_SALIDA.endswith('.xlsx'):
    df_input.to_excel(ARCHIVO_SALIDA, index=False)
else:
    df_input.to_csv(ARCHIVO_SALIDA, index=False)

print(f"¡Listo! Guardado en {ARCHIVO_SALIDA}")

--- INICIANDO ---
-> Leyendo nombres.csv | Separador: ';'
Filas leídas en Input: 1899
Generando llaves...
Cargando Maestra...
-> Leyendo ruts.csv | Separador: ','
Indexando maestra...
Cruzando 1899 registros...



Progreso: 100%|███████████████████████████| 1899/1899 [1:46:58<00:00,  3.38s/it]


¡Listo! Guardado en resultado_cruce_final.xlsx


In [ ]:
print("Iniciando busqueda difusa...")

resultados = {
    'RUT_ENCONTRADO': [],
    'SCORE': [],
    'NOMBRE_MATCH': []
}
for c in COLUMNAS_EXTRA:
    resultados[c] = []

# Iteracion con barra de progreso
for nombre_buscar in tqdm(df_input['__key__'], desc="Progreso"):
    
    match_row = None
    score = 0
    nombre_match = None

    if nombre_buscar:
        # Busqueda del mejor candidato
        res = process.extractOne(
            nombre_buscar,
            opciones_nombres,
            scorer=fuzz.token_sort_ratio,
            score_cutoff=UMBRAL
        )
        
        if res:
            nombre_match, score, _ = res
            # Recuperamos datos usando el indice pre-calculado
            idx = mapa_indices[nombre_match]
            match_row = df_fuente.loc[idx]

    # Asignacion de resultados
    val_rut = match_row[COL_RUT_BUSQUEDA] if match_row is not None else None
    resultados['RUT_ENCONTRADO'].append(val_rut)
    resultados['SCORE'].append(score)
    resultados['NOMBRE_MATCH'].append(nombre_match)
    
    for c in COLUMNAS_EXTRA:
        val = match_row[c] if match_row is not None and c in match_row else None
        resultados[c].append(val)

print("Cruce finalizado.")

In [ ]:
# Integrar resultados al DF original
for k, v in resultados.items():
    df_input[k] = v

# Eliminar columna auxiliar
if '__key__' in df_input.columns:
    df_input.drop(columns=['__key__'], inplace=True)

print(f"Guardando en {ARCHIVO_SALIDA}...")
if ARCHIVO_SALIDA.endswith('.xlsx'):
    df_input.to_excel(ARCHIVO_SALIDA, index=False)
else:
    df_input.to_csv(ARCHIVO_SALIDA, index=False)

print("Proceso completado.")